In [2]:
# If cloning is needed -- i.e., if hangman.py and word counts are not already
# available in file, you can run this
!git clone https://github.com/ucsd-cse150a-w25/hw1.git
!cp hw1/hangman.py hangman.py
!cp hw1/hw1_word_counts_05.txt hw1_word_counts_05.txt

fatal: destination path 'hw1' already exists and is not an empty directory.


In [3]:
from hangman import hangman_game

# Play the game by yourself
hangman_game()



  +---+
  O   |
 /|\  |
 / \  |
     ===

Word: I _ _ E _
Tried letters: A, E, I, O, R, S, T, W
Game over! The word was:  INDEX


0

In [6]:
import random

def random_inference(
    letters_tried: set[str],
    word_pattern: list[str],
    word_counts: dict[str, int]
) -> str:
    '''
    Random inference for playing hangman. This should be a simple method which returns a letter
    that is NOT in letters_tried but any other letter, at random.

    Hint: use the random.choice method
    '''
    # TODO: Implement random inference code to guess hangman
    alphabet = set("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
    remaining_letters = list(alphabet - letters_tried)
    return random.choice(remaining_letters) if remaining_letters else ''

hangman_game(random_inference)



  +---+
  O   |
 /|\  |
 / \  |
     ===

Word: _ _ _ _ S
Tried letters: B, G, I, J, P, S, U
Game over! The word was:  RATES


0

In [13]:
import collections

def bayesian_inference(
    letters_tried: set[str],
    word_pattern: list[str],
    word_counts: dict[str, int]
) -> str:
    '''
    Bayesian inference method for playing hangman. The parameters given are as follows:

    - letters_tried (set[str]): A set of strings which consist of all the letters that have already
        been tried. For example, if 'A', 'E' has been guessed, `letters_tried = {'A', 'E'}`
    - word_pattern (list[str]): A list of single characters that describe the current guess state.
        For example, if the hangman state is _AB__, `word_pattern = ['_', 'A', 'B', '_', '_']`
    - word_counts (dict[str, int]): The word counts dictionary which contains all possible 5 letter
        words in our hangman game and their respective counts. For example, a key value pair could
        be 'AARON': 413.

    Return type: a single character string as your next best guess.
    '''
    # TODO: Implement inference code to play hangman optimally
    possible_words = {
        word: count for word, count in word_counts.items()
        if all(
            (char == '_' or word[i] == char)
            and (char == '_' or word[i] not in letters_tried)
            for i, char in enumerate(word_pattern)
        )
    }

    letter_frequencies = collections.Counter()
    for word, count in possible_words.items():
        for i, letter in enumerate(word):
            if word_pattern[i] == '_' and letter not in letters_tried:
                letter_frequencies[letter] += count

    if letter_frequencies:
        return max(letter_frequencies, key=letter_frequencies.get)

    remaining_letters = list(set("ABCDEFGHIJKLMNOPQRSTUVWXYZ") - letters_tried)
    return random.choice(remaining_letters) if remaining_letters else 'E'

# Run the game
hangman_game(bayesian_inference)



  +---+
  O   |
 /|\  |
 / \  |
     ===

Word: Y E _ _ S
Tried letters: C, D, E, I, J, P, S, X, Y
Game over! The word was:  YEARS


0

In [1]:
from typing import Optional, Callable
from tqdm import tqdm
import time

def benchmark(
    inference: Optional[Callable],
    num_runs: int = 1000,
    seed: int = 0,
    timeout: float = 5.0
) -> None:
    '''
    Benchmark method for testing out the bayesian inference method. The parameters given are:

    - inference: The function that should match the bayesian_inference() method above.
    - seed: The seed to pass into the hangman_game function.

    Return type: none.

    Print the accuracy out of num_runs. If the function throws an error, it should count as a fail.
    '''
    # TODO: Implement benchmark testing for a given function
    # Hint: use `random.seed(seed)` ONCE at the start
    # Hint: pass `interactive = False` into the hangman_game function to run faster without outputs.
    # Optional: use the `tqdm` module to keep track of the evaluation progress.

    random.seed(seed)
    successes = 0

    for i in tqdm(range(num_runs), desc=f"Benchmarking {inference.__name__}"):
        try:
            start_time = time.time()
            success = hangman_game(inference, interactive=False)
            elapsed_time = time.time() - start_time

            if elapsed_time > timeout:
                print(f"Timeout: Game {i+1} took {elapsed_time:.2f}s. Skipping...")
                continue

            if success:
                successes += 1

        except Exception as e:
            print(f"Error in game {i+1}: {e}")

    accuracy = (successes / num_runs) * 100
    print(f"{inference.__name__} Accuracy: {successes}/{num_runs} ({accuracy:.2f}%)")

benchmark(random_inference)
benchmark(bayesian_inference)

NameError: name 'random_inference' is not defined